# Timing and profiling matvec_real functions

This code uses the magic functions `%timeit` and `%lprun` <sup> 1 </sup> to measure, respectively, the total and the line-by-line excetution times of the functions `matvec_real_dumb` and `matvec_real_numba`.

* <sup> 1 </sup>[Profiling and Timing Code - excerpt from the Python Data Science Handbook by Jake VanderPlas](https://jakevdp.github.io/PythonDataScienceHandbook/01.07-timing-and-profiling.html)

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import functions as fcs
from tqdm.notebook import tqdm as tq # produce the progress bar

In [7]:
from numba import njit, prange, guvectorize

In [2]:
# needed to use the magic function %lprun
%load_ext line_profiler

In [ ]:
N = np.array([10, 30, 50, 70, 100, 300, 500, 700, 1000])

In [ ]:
list_time_dumb = []
list_time_numba = []
list_time_numpy = []
for Ni in tq(N):
    vector = np.ones(Ni)
    matrix = np.ones((Ni,Ni))
    # matvec_real_dumb
    time = %timeit -o fcs.matvec_real_dumb(matrix, vector)
    list_time_dumb.append(time.average)
    # matvec_real_numba
    time = %timeit -o fcs.matvec_real_numba(matrix, vector)
    list_time_numba.append(time.average)
    # numpy.dot
    time = %timeit -o np.dot(matrix, vector)
    list_time_numpy.append(time.average)

In [ ]:
plt.figure(figsize=(10,7))
plt.plot(N, np.asarray(list_time_dumb)*1e6, 'bo-', label = 'dumb')
plt.plot(N, np.asarray(list_time_numba)*1e6, 'go-', label = 'numba')
plt.plot(N, np.asarray(list_time_numpy)*1e6, 'ro-', label = 'numpy')
plt.legend(loc = 'best', fontsize = 14)
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
plt.xlabel('N', fontsize = 16)
plt.ylabel('Time ($\mu$s)', fontsize = 16)
plt.yscale('log')
plt.grid()
plt.show()

### line-by-line profiling with `%lprun`

In [3]:
vector = np.ones(500)
matrix = np.ones((500,500))

In [4]:
def matvec_real_dot(A, x):
    result = np.zeros(A.shape[0])
    for i in range(A.shape[0]):
        result[i] = fcs.dot_real_numba(A.real[i], x.real)
    return result

In [5]:
def matvec_real_sum(A, x):
    result = A*x
    result = np.sum(result, axis=1)
    return result

In [6]:
def matvec_real_reduce(A, x):
    result = A*x
    result = np.add.reduce(result, axis=1)
    return result

In [ ]:
%timeit np.dot(matrix, vector)

In [ ]:
%timeit fcs.matvec_real_dumb(matrix, vector)

In [ ]:
%timeit fcs.matvec_real_numba(matrix, vector)

In [ ]:
%timeit matvec_real_dot(matrix, vector)

In [ ]:
%timeit matvec_real_sum(matrix, vector)

In [ ]:
%timeit matvec_real_reduce(matrix, vector)

In [ ]:
%lprun -f fcs.matvec_real_dumb fcs.matvec_real_dumb(matrix, vector)

In [ ]:
%lprun -f matvec_real_dot matvec_real_dot(matrix, vector)

In [ ]:
%lprun -f matvec_real_sum matvec_real_sum(matrix, vector)

In [ ]:
%lprun -f matvec_real_reduce matvec_real_reduce(matrix, vector)

In [ ]:
@njit
def Ax_jit(A, x, res):
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            res[i,j] = A.real[i,j]*x.real[j]
    return res

In [ ]:
result = np.zeros_like(matrix)

In [ ]:
%timeit matrix*vector

In [ ]:
%timeit Ax_jit(matrix, vector, result)

In [ ]:
def matvec_real_numba(A, x):
    result = np.empty_like(A)
    result = Ax_jit(A, x, result)
    result = np.add.reduce(result, axis=1)
    return result

In [ ]:
%timeit matvec_real_numba(matrix, vector)

In [ ]:
%lprun -f matvec_real_numba matvec_real_numba(matrix, vector)